In [1]:
import graphlab
import multiprocessing
cpu_count = multiprocessing.cpu_count()
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', cpu_count)

This non-commercial license of GraphLab Create for academic use is assigned to vung.pham@ttu.edu and will expire on June 18, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1538236721.log


In [2]:
#load data
f = graphlab.SFrame("avocado.csv")
processedf = f

Finished parsing file /Users/idvlab/Documents/TTU/htdocs/Outliagnostics.github.io/data/avocadoprice/avocado.csv

Parsing completed. Parsed 100 lines in 0.094615 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,float,float,float,float,float,float,float,float,str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/idvlab/Documents/TTU/htdocs/Outliagnostics.github.io/data/avocadoprice/avocado.csv

Parsing completed. Parsed 18249 lines in 0.051682 secs.

In [3]:
f

X1,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49
2,2015-12-13,0.93,118220.22,794.7,109149.67,130.5,8145.35,8042.21,103.14
3,2015-12-06,1.08,78992.15,1132.0,71976.41,72.58,5811.16,5677.4,133.76
4,2015-11-29,1.28,51039.6,941.48,43838.39,75.78,6183.95,5986.26,197.69
5,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44
6,2015-11-15,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05
7,2015-11-08,0.98,109428.33,703.75,101815.36,80.0,6829.22,6266.85,562.37
8,2015-11-01,1.02,99811.42,1022.15,87315.57,85.34,11388.36,11104.53,283.83
9,2015-10-25,1.07,74338.76,842.4,64757.44,113.0,8625.92,8061.47,564.45


In [3]:
#date time
from datetime import datetime
def parseDate(dateStr):
    dateFrm = '%Y-%m-%d'
    if ":" in dateStr:
        dateFrm = "%Y-%m-%d %H:%M:%S"
    return datetime.strptime(dateStr, dateFrm);
def monthDiff(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

#process date time
processedf['date'] = processedf['date'].apply(parseDate)

In [4]:
minDate = min(processedf['date'])
def monthIndex(d):
    return monthDiff(d, minDate);

processedf['monthIndex'] = processedf['date'].apply(monthIndex)

In [5]:
#processing variables
VariablesOrig = ["open", "close", "volume"]
Variables = ["openDiffOverVolume", "closeDiffOverVolume"]

In [6]:
#aggregate data
import graphlab.aggregate as agg

In [7]:
result = processedf.groupby(key_columns=['symbol', 'monthIndex'], operations={VariablesOrig[0]: agg.MEAN(VariablesOrig[0]), VariablesOrig[1]: agg.MEAN(VariablesOrig[1]), VariablesOrig[2]:agg.MEAN(VariablesOrig[2])})

In [8]:
result

monthIndex,symbol,volume,close,open
81,MCHP,1679871.42857,60.3523808571,60.4623805238
34,PLD,2485785.71429,33.7133332857,33.7338095238
65,K,1773418.18182,62.492727,62.4513632727
80,COTY,14997276.1905,24.2971427143,24.5609522381
27,CBS,7547685.0,32.9535001,32.92300015
75,CELG,4915866.66667,106.390476429,106.244285619
18,BSX,19514915.0,7.1145,7.1045
7,MCK,2555736.36364,61.2499998636,61.3313637273
73,SEE,2236355.0,43.04400025,42.982
74,PYPL,8605163.63636,39.3545455909,39.2309092273


In [10]:
from joblib import Parallel, delayed
import multiprocessing

In [11]:
cpu_count = multiprocessing.cpu_count()

In [12]:
cpu_count

4

In [25]:
type(result)

graphlab.data_structures.sframe.SFrame

In [13]:
result1 = list(result)

In [14]:
inputs = range(1, len(result1), 1)
def processInput0(i):
    return (result1[i][VariablesOrig[0]] - result1[i-1][VariablesOrig[0]])/result1[i][VariablesOrig[2]]
def processInput1(i):
    return (result1[i][VariablesOrig[1]] - result1[i-1][VariablesOrig[1]])/result1[i][VariablesOrig[2]]

In [ ]:
data0 = Parallel(n_jobs=cpu_count)(delayed(processInput0)(i) for i in inputs)

/Users/idvlab/anaconda2/lib/python2.7/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
data1 = Parallel(n_jobs=cpu_count)(delayed(processInput1)(i) for i in inputs)

In [12]:
result[Variables[0]] = graphlab.SFrame({Variables[0]: data0})[Variables[0]]
result[Variables[1]] = graphlab.SFrame({Variables[1]: data1})[Variables[1]]

In [13]:
Scagnostics = ["Outlying"]
Symbols = result['symbol'].unique()

In [14]:
def getDataFromMonth(month):
    r = result.filter_by([month], 'monthIndex')
    maxV0 = max(r[Variables[0]])
    minV0 = min(r[Variables[0]])
    maxV1 = max(r[Variables[1]])
    minV1 = min(r[Variables[1]])
    r['normalized'+Variables[0]] = r[Variables[0]].apply(lambda c: (c-minV0)/(maxV0-minV0))
    r['normalized'+Variables[1]] = r[Variables[1]].apply(lambda o: (o-minV1)/(maxV1-minV1))
    return list(r)

In [15]:
monthSymbolData = []
for month in range(max(result['monthIndex'])+1):
    monthSymbolData = monthSymbolData + getDataFromMonth(month)

In [16]:
monthSymbolDataObject = {}
for item in monthSymbolData:
    monthSymbolDataObject[item['symbol']+str(item['monthIndex'])] = item

In [17]:
def getDataMonthSymbol(month, symbol):
    return monthSymbolDataObject.get(symbol+str(month), None)

In [18]:
processedMonthData = []
for month in range(max(result['monthIndex'])+1):
    s0 = []
    s1 = []
    for symbol in Symbols:
        d = getDataMonthSymbol(month, symbol)
        if d!=None:
            s0.append(d['normalized'+Variables[0]])
            s1.append(d['normalized'+Variables[1]])
        else:
            s0.append('NaN')
            s1.append('NaN')
    processedMonthData.append({'s0': s0, 's1': s1})

In [19]:
Scagnostics0 = [0];
for md in processedMonthData:
    md['Scagnostics0'] = Scagnostics0

In [4]:
processedSymbolData = {}
for symbol in Symbols:
    singleSymbolData = []
    for month in range(max(result['monthIndex'])+1):
        d = getDataMonthSymbol(month, symbol)
        if d!=None:
            v0 = d[Variables[0]]
            v1 = d[Variables[1]]
            s0 = d['normalized'+Variables[0]]
            s1 = d['normalized'+Variables[1]]
        else:
            v0 = 'NaN'
            v1 = 'NaN'
            s0 = 'NaN'
            s1 = 'NaN'
        singleMonthData ={"v0": int(v0), "v1": int(v1), "s0": s0, "s1": s1, "Outlying": 0, "year": month}
        singleSymbolData.append(singleMonthData)
    processedSymbolData[symbol] = singleSymbolData

NameError: name 'Symbols' is not defined

In [23]:
finalResult = {}
finalResult['Scagnostics'] = Scagnostics
finalResult['Variables'] = Variables
finalResult['Countries'] = list(Symbols)
finalResult['YearsData'] = processedMonthData
finalResult['CountriesData'] = processedSymbolData

import json
with open('pricesDiffOverVolume.json', 'w') as outfile:
    json.dump(finalResult, outfile)

In [22]:
processedMonthData

[{'Scagnostics0': [0],
  's0': [0.7429452687321004,
   0.6959150470312363,
   0.7069442239797581,
   0.7052799987990251,
   0.7039539343227792,
   0.7103002464757923,
   0.7058048284098093,
   'NaN',
   0.7261732870337004,
   0.6987288343334859,
   'NaN',
   0.699331130602181,
   0.7037079380818267,
   0.7048662854100399,
   0.6930627771961663,
   0.6946206582982418,
   0.7039255710260863,
   0.700776683567157,
   0.7022179455942568,
   0.7034444349580136,
   0.6531695407726288,
   0.7061763703404798,
   0.7046975565707253,
   0.7025766697275858,
   0.704150072596472,
   0.7042422289792415,
   0.7115447447921714,
   0.705149939172479,
   0.7033350343789694,
   0.7052344619868639,
   0.6974324540501301,
   0.7017661989787236,
   0.6998393526523192,
   'NaN',
   0.7068689403614632,
   0.7046420805024625,
   0.7029629569518374,
   0.7042792300023579,
   0.7041346755793282,
   0.7057351156250492,
   0.7025509708558005,
   0.7320395669021051,
   0.7061943238481447,
   0.7015753645813148,
  